In [1]:
import pandas as pd
import logging

from split_data_type import SplitDataType
from pipeline import TreeSimilarPipeline
from feature_manager import FeatureManager

In [2]:
logging.basicConfig(level=logging.DEBUG)

In [3]:
target_column = "SalePrice"

train_df = pd.read_csv("../HousePrice_train.csv")
train_X = train_df.drop(columns=[target_column])
train_y = train_df[target_column]
test_X = pd.read_csv("../HousePrice_test.csv")
test_y = pd.read_csv("../sample_submission.csv")[target_column]

In [4]:
FeatureManager.get_smallest_category(train_X)

,value,frequency(%)
column,,
Utilities,NoSeWa,0.068493
RoofMatl,Metal,0.068493
Exterior1st,AsphShn,0.068493
Condition2,PosA,0.068493
HeatingQC,Po,0.068493
Heating,Floor,0.068493
ExterCond,Po,0.068493
Exterior2nd,Other,0.068493
Functional,Sev,0.068493


In [5]:
FeatureManager.get_high_entropy(train_X)

['Id']

# Досліджую фічі, де є маленький % значень

In [6]:
# Мусор фіча, в смітник
train_X['Utilities'].value_counts() 

Utilities
AllPub    1459
NoSeWa       1
Name: count, dtype: int64

In [7]:
# Мусор фіча
train_X['RoofMatl'].value_counts() 

RoofMatl
CompShg    1434
Tar&Grv      11
WdShngl       6
WdShake       5
Metal         1
Membran       1
Roll          1
ClyTile       1
Name: count, dtype: int64

In [8]:
# перетворю < 200 в others
train_X['Exterior1st'].value_counts() 

Exterior1st
VinylSd    515
HdBoard    222
MetalSd    220
Wd Sdng    206
Plywood    108
CemntBd     61
BrkFace     50
WdShing     26
Stucco      25
AsbShng     20
BrkComm      2
Stone        2
AsphShn      1
ImStucc      1
CBlock       1
Name: count, dtype: int64

In [9]:
# мусорка
train_X['Condition2'].value_counts() 

Condition2
Norm      1445
Feedr        6
Artery       2
RRNn         2
PosN         2
PosA         1
RRAn         1
RRAe         1
Name: count, dtype: int64

In [10]:
# Зведу всі оціночні категорії в positive, neutral, negative
train_X['HeatingQC'].value_counts() 

HeatingQC
Ex    741
TA    428
Gd    241
Fa     49
Po      1
Name: count, dtype: int64

In [11]:
# мусор
train_X['Heating'].value_counts() 

Heating
GasA     1428
GasW       18
Grav        7
Wall        4
OthW        2
Floor       1
Name: count, dtype: int64

In [12]:
train_X['ExterCond'].value_counts() 

ExterCond
TA    1282
Gd     146
Fa      28
Ex       3
Po       1
Name: count, dtype: int64

In [13]:
# < 100 в others
train_X['Exterior2nd'].value_counts() 

Exterior2nd
VinylSd    504
MetalSd    214
HdBoard    207
Wd Sdng    197
Plywood    142
CmentBd     60
Wd Shng     38
Stucco      26
BrkFace     25
AsbShng     20
ImStucc     10
Brk Cmn      7
Stone        5
AsphShn      3
Other        1
CBlock       1
Name: count, dtype: int64

In [14]:
# в is_typical_functional
train_X['Functional'].value_counts() 

Functional
Typ     1360
Min2      34
Min1      31
Mod       15
Maj1      14
Maj2       5
Sev        1
Name: count, dtype: int64

In [15]:
# в is_standard_electrical
train_X['Electrical'].value_counts() 

Electrical
SBrkr    1334
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: count, dtype: int64

In [16]:
train_X['Neighborhood'].value_counts() 

Neighborhood
NAmes      225
CollgCr    150
OldTown    113
Edwards    100
Somerst     86
Gilbert     79
NridgHt     77
Sawyer      74
NWAmes      73
SawyerW     59
BrkSide     58
Crawfor     51
Mitchel     49
NoRidge     41
Timber      38
IDOTRR      37
ClearCr     28
SWISU       25
StoneBr     25
Blmngtn     17
MeadowV     17
BrDale      16
Veenker     11
NPkVill      9
Blueste      2
Name: count, dtype: int64

In [17]:
# в is_gable_roof_style
train_X['RoofStyle'].value_counts() 

RoofStyle
Gable      1141
Hip         286
Flat         13
Gambrel      11
Mansard       7
Shed          2
Name: count, dtype: int64

In [18]:
# в is_norm_condition1
train_X['Condition1'].value_counts() 

Condition1
Norm      1260
Feedr       81
Artery      48
RRAn        26
PosN        19
RRAe        11
PosA         8
RRNn         5
RRNe         2
Name: count, dtype: int64

In [19]:
# в new, warrante_deed, others
train_X['SaleType'].value_counts() 

SaleType
WD       1267
New       122
COD        43
ConLD       9
ConLI       5
ConLw       5
CWD         4
Oth         3
Con         2
Name: count, dtype: int64

In [20]:
train_X['BsmtCond'].value_counts() 

BsmtCond
TA    1311
Gd      65
Fa      45
Po       2
Name: count, dtype: int64

In [21]:
#raise Exception("Stop here to check the features")

In [22]:
train_X = TreeSimilarPipeline(train_X, SplitDataType.TRAIN).build()

DEBUG:root:TRAIN - Function: _drop_not_needed
Size: (1460, 75)
Columns with null: LotFrontage, Alley, MasVnrType, MasVnrArea, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, Electrical, FireplaceQu, GarageType, GarageYrBlt, GarageFinish, GarageQual, GarageCond, PoolQC, Fence, MiscFeature

DEBUG:root:TRAIN - Function: _fill_null
Size: (1460, 75)
Columns with null: LotFrontage, Alley, MasVnrType, MasVnrArea, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, Electrical, FireplaceQu, GarageType, GarageYrBlt, GarageFinish, GarageQual, GarageCond, PoolQC, Fence, MiscFeature

DEBUG:root:TRAIN - Function: _encode
Size: (1460, 75)
Columns with null: LotFrontage, Alley, MasVnrType, MasVnrArea, BsmtQual, BsmtExposure, BsmtFinType1, BsmtFinType2, FireplaceQu, GarageType, GarageYrBlt, GarageFinish, GarageQual, GarageCond, PoolQC, Fence, MiscFeature

DEBUG:root:TRAIN - Function: _normalize
Size: (1460, 75)
Columns with null: LotFrontage, Alley, MasVnrType, MasVnrArea, BsmtQ

In [23]:
train_X.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,...,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Is_typical_functional,Is_standard_electrical,Is_gable_roof_style,Is_norm_condition1
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,Inside,Gtl,...,NaN,0,2,2008,Warranty_deed,Normal,1,1,1,1
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,FR2,Gtl,...,NaN,0,5,2007,Warranty_deed,Normal,1,1,1,0
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,Inside,Gtl,...,NaN,0,9,2008,Warranty_deed,Normal,1,1,1,1
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,Corner,Gtl,...,NaN,0,2,2006,Warranty_deed,Abnorml,1,1,1,1
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,FR2,Gtl,...,NaN,0,12,2008,Warranty_deed,Normal,1,1,1,1


In [24]:
train_X['BsmtCond'].value_counts() 

BsmtCond
neutral     1311
positive      65
negative      47
Others        37
Name: count, dtype: int64